In [134]:
import pandas as pd
import numpy as np
import h5py

In [135]:
# This return the maximum value in a given nd.array
def get_max_list(a_list):
    # Converts the array to a dataframe
    df = pd.DataFrame(a_list)
    max_values = df.max()
    return max_values

In [136]:
# Read the RAS HDF File Folder path
ras_hdf_file = r"C:\Users\richmond\AMH Philippines, Inc\PP23.307 Rockwell and Roll - My Documents\Python\amh_pyqgis\flow_extraction\RAS\lagnas.p01.hdf"

In [137]:
# DataFrame Column Name
column = [
    'Reference Line',
    'Discharge',
    'Flow Velocity',
    'Flow Area',
    'WSE',
    'EGL'
]


"""
This is the complete column names:

column = [
    'Reference Line',
    'Discharge',
    'Flow Velocity',
    'Flow Area',
    'WSE',
    'EGL',
    'Thalweg',
    'LOB',
    'ROB',
]

"""

"\nThis is the complete column names:\n\ncolumn = [\n    'Reference Line',\n    'Discharge',\n    'Flow Velocity',\n    'Flow Area',\n    'WSE',\n    'EGL',\n    'Thalweg',\n    'LOB',\n    'ROB',\n]\n\n"

In [138]:
# Read the HDF5 File
f =  h5py.File(ras_hdf_file, 'r')

In [139]:
# Get the names fo the reference line names
ref_name_path = r'/Geometry/Reference Lines/Attributes'
ref_names = np.array(f[ref_name_path])
ref_col_names = [x[0].decode('utf-8') for x in ref_names]

In [140]:
# Get the max discharge per reference lines
ref_line_path = r'/Results/Unsteady/Output/Output Blocks/DSS Hydrograph Output/Unsteady Time Series/Reference Lines'
ref_flow_array = f[ref_line_path]['Flow']
flow_max_values = np.max(ref_flow_array, axis=0)
ref_flow = flow_max_values.tolist()

In [141]:
# Get the Max Velocity per Reference Lines
ref_vel_array = f[ref_line_path]['Velocity']
vel_max_values = np.max(ref_vel_array, axis=0)
ref_velocity = vel_max_values.tolist()

In [142]:
# Compute the Max Flow Area
# Given by the Eq. Q = A * V; A = Q / V
ref_max_flow_area_array = np.divide(
                            ref_flow_array,
                            ref_vel_array,
                            )
ref_max_flow_area = np.nanmax(ref_max_flow_area_array, axis=0).tolist()

C:\Users\richmond\AppData\Local\Temp\ipykernel_21072\2517206911.py:3: RuntimeWarning: invalid value encountered in divide
  ref_max_flow_area_array = np.divide(


In [143]:
# Get the Max WSE
ref_wse_array = f[ref_line_path]['Water Surface']
ref_wse = np.max(ref_wse_array, axis=0).tolist()

In [144]:
# Compute the EGL
# Given by the equation: EGL = WSE + V^2 / 2g (g=9.81 m/s^2)
g = 9.81 * 2 
vel_head = np.power(ref_vel_array, 2) / g
egl_array = ref_wse + vel_head
ref_egl = np.max(egl_array, axis=0).tolist()

In [145]:
# Create a DataFrame
df = pd.DataFrame({
    'Reference Line' : ref_col_names,
    'Discharge' : ref_flow,
    'Flow Velocity' : ref_velocity,
    'Flow Area': ref_max_flow_area,
    'WSE': ref_wse,
    'EGL': ref_egl
})

In [146]:
f.close()